In [28]:
%load_ext autoreload
%autoreload 2
%pylab inline

[autoreload of abstraction_validation.sm_paths failed: Traceback (most recent call last):
  File "/home/aremath/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/aremath/.local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 786, in exec_module
  File "<frozen importlib._bootstrap_external>", line 923, in get_code
  File "<frozen importlib._bootstrap_external>", line 853, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "../abstraction_validation/sm_paths.py", lin

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/home/aremath/.local/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [29]:
import sys
sys.path.append("..")

In [30]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [31]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [43]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [44]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")
parsed_rom = rom.parse()

In [45]:
all_posns = all_positions(rooms, parsed_rom, is_global=False)

Landing_Site 0x791f8
	 Extra: []
	 Missing: ['Landing_Site_Ship']
West_Ocean 0x793fe
	 Extra: ['West_Ocean_L', 'West_Ocean_R4']
	 Missing: ['West_Ocean_R6', 'West_Ocean_L2']
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: ['Bomb_Torizo_Bombs']
Spore_Spawn 0x79dc7
	 Extra: []
	 Missing: ['Spore_Spawn_Spore_Spawn']
Kraid 0x7a59f
	 Extra: []
	 Missing: ['Kraid_Kraid']
Statues 0x7a66a
	 Extra: []
	 Missing: ['Statues_Statues']
Crocomire 0x7a98d
	 Extra: []
	 Missing: ['Crocomire_Crocomire']
Water_Closet 0x7b1e5
	 Extra: []
	 Missing: ['Water_Closet_Drain']
Golden_Torizo 0x7b283
	 Extra: []
	 Missing: ['Golden_Torizo_Golden_Torizo']
Ridley 0x7b32e
	 Extra: []
	 Missing: ['Ridley_Ridley']
Phantoon 0x7cd13
	 Extra: []
	 Missing: ['Phantoon_Phantoon']
West_Sand_Hall 0x7d461
	 Extra: []
	 Missing: ['West_Sand_Hall_TS']
East_Sand_Hall 0x7d4c2
	 Extra: []
	 Missing: ['East_Sand_Hall_TS']
West_Sand_Hole 0x7d4ef
	 Extra: []
	 Missing: ['West_Sand_Hole_TS']
East_Sand_Hole 0x7d51e
	 Extra: 

In [46]:
import omega
from omega.symbolic.fol import Context

In [47]:
#all_posns

In [48]:
#design["Rooms"]["Botwoon_Energy"]

In [49]:
#all_global_positions

In [50]:
#design["Items"], design["Bosses"]

In [87]:
room_addrs = {name: room.mem_address & 0xffff for name, room in rooms.items()}

In [51]:
def loc(room_name, node_name, when="prev"):
    #TODO: local positions
    #TODO: with room_prev and room_next
    try:
        node_loc = all_posns[f"{room_name}_{node_name}"]
    except KeyError:
        return "FALSE"
    xx = f"x_{when} = {int(node_loc.x)}"
    yy = f"y_{when} = {int(node_loc.y)}"
    #TODO room_id
    return " & ".join([xx, yy])

def item_transitions(item_gained=None):
    if item_gained is None:
        return "(items_unchanged)"
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = f"{i}_prev < {i}_next"
        else:
            clause = f"{i}_prev = {i}_next"
        clauses.append(clause)
    return " & ".join(clauses)

def required_itemsets(itemsets):
    return "(" + " | ".join([" & ".join([f"{item}_prev = 1" for item in itemset]) for itemset in itemsets]) + ")"


In [52]:
from tqdm.notebook import tqdm

In [89]:
items_unchanged_def = "items_unchanged == " + " & ".join([f"{i}_prev = {i}_next" for i in design["Items"] | design["Bosses"]])
room_unchanged_def = "room_unchanged == room_id_prev = room_id_next"

In [90]:
items_unchanged_def

'items_unchanged == B_prev = B_next & PB_prev = PB_next & SPB_prev = SPB_next & S_prev = S_next & M_prev = M_next & G_prev = G_next & SA_prev = SA_next & V_prev = V_next & GS_prev = GS_next & SB_prev = SB_next & HJ_prev = HJ_next & MB_prev = MB_next & CB_prev = CB_next & WB_prev = WB_next & E_prev = E_next & PLB_prev = PLB_next & IB_prev = IB_next & SJ_prev = SJ_next & Spazer_prev = Spazer_next & RT_prev = RT_next & XR_prev = XR_next & Drain_prev = Drain_next & Shaktool_prev = Shaktool_next & START_prev = START_next & Statues_prev = Statues_next & Kraid_prev = Kraid_next & Phantoon_prev = Phantoon_next & Draygon_prev = Draygon_next & Ridley_prev = Ridley_next & Mother_Brain_prev = Mother_Brain_next & Botwoon_prev = Botwoon_next & Spore_Spawn_prev = Spore_Spawn_next & Golden_Torizo_prev = Golden_Torizo_next & Bomb_Torizo_prev = Bomb_Torizo_next & Crocomire_prev = Crocomire_next'

In [92]:
doors = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        links.append(loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions() + " & room_unchanged")
        if node_name in room['Drops']:
            links.append(loc(room_name, node_name) + " & " + loc(room_name, node_name, when="next") + " & " + item_transitions(room['Drops'][node_name]) + " & room_unchanged")
    for node_name, door in room['Doors'].items():
        a1 = room_addrs[room_name]
        a2 = room_addrs[door["Room"]]
        doors.append(loc(room_name, node_name) + " & " + loc(door['Room'], door['Node'], when="next") + " & " + item_transitions() + f" & room_id_prev = {a1} & room_id_next = {a2}")
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            links.append(loc(room_name, node_name) + " & " + loc(room_name, other_node_name, when="next") + " & " + required_itemsets(edge['Requirements']) + " & " + item_transitions() + " & room_unchanged")
    link_str = " | ".join(["(" + l + ")" for l in links])
    fname = f"../output/tla/{room_name}.tla"
    with open(fname, "w") as f:
        f.write(f"LET {items_unchanged_def} IN LET {room_unchanged_def} in {link_str}")
door_fname = f"../output/tla/doors.tla"
#door_str = " | ".join(doors)
door_str = " | ".join(doors)
with open(door_fname, "w") as f:
    f.write(f"LET {items_unchanged_def} IN ({door_str})")

  0%|          | 0/255 [00:00<?, ?it/s]

In [61]:
from omega.logic.lexyacc import Parser

In [62]:
p = Parser()

In [63]:
%%time
with open("../output/tla/doors.tla") as f:
    s = f.read()
    p.parse(s)

CPU times: user 68.9 ms, sys: 0 ns, total: 68.9 ms
Wall time: 71 ms


In [59]:
p.parse("LET x == 7 IN y")

Operator('LET', [Binary('==', Terminal('x', 'opname'), Num('7', 'num'))], Var('y', 'var'))

In [71]:
hex(rooms["Landing_Site"].mem_address)

'0x791f8'

In [86]:
hex(room_addrs["Landing_Site"])

'0x91f8'